# Rebuild from Corners Technique

Implement the chistyakov kernel technique:
- For each large N layout, check all 4 corners
- Sort trees by distance from corner
- Extract subsets to find better solutions for smaller N

In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 25

class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (float(Decimal('0.0')), float(tip_y)),
            (float(top_w / Decimal('2')), float(tier_1_y)),
            (float(top_w / Decimal('4')), float(tier_1_y)),
            (float(mid_w / Decimal('2')), float(tier_2_y)),
            (float(mid_w / Decimal('4')), float(tier_2_y)),
            (float(base_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(base_y)),
            (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),
            (float(-(trunk_w / Decimal('2'))), float(base_y)),
            (float(-(base_w / Decimal('2'))), float(base_y)),
            (float(-(mid_w / Decimal('4'))), float(tier_2_y)),
            (float(-(mid_w / Decimal('2'))), float(tier_2_y)),
            (float(-(top_w / Decimal('4'))), float(tier_1_y)),
            (float(-(top_w / Decimal('2'))), float(tier_1_y)),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1])))

def get_total_score(dict_of_side_length):
    score = Decimal('0')
    for k, v in dict_of_side_length.items():
        score += v ** 2 / Decimal(str(int(k)))
    return score

print("Classes defined.")

Classes defined.


In [2]:
def parse_csv(csv_path):
    """Parse CSV and return dict of tree lists and side lengths."""
    result = pd.read_csv(csv_path)
    result['x'] = result['x'].str.strip('s')
    result['y'] = result['y'].str.strip('s')
    result['deg'] = result['deg'].str.strip('s')
    result[['group_id', 'item_id']] = result['id'].str.split('_', n=2, expand=True)

    dict_of_tree_list = {}
    dict_of_side_length = {}
    for group_id, group_data in result.groupby('group_id'):
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg']) 
                     for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
        dict_of_side_length[group_id] = get_tree_list_side_length(tree_list)

    return dict_of_tree_list, dict_of_side_length

def copy_layout(layout):
    return [tree.clone() for tree in layout]

print("Parse functions defined.")

Parse functions defined.


In [3]:
# Load baseline solution
baseline_path = '/home/submission/submission.csv'
dict_of_tree_list, dict_of_side_length = parse_csv(baseline_path)

# Calculate baseline score
baseline_score = get_total_score(dict_of_side_length)
print(f'Baseline score: {float(baseline_score):.6f}')
print(f'Number of layouts: {len(dict_of_tree_list)}')

Baseline score: 70.659958
Number of layouts: 200


In [4]:
def rebuild_from_corners(dict_of_tree_list, dict_of_side_length, verbose=True):
    """
    For each large N layout, check all 4 corners.
    Sort trees by distance from corner.
    Extract subsets to find better solutions for smaller N.
    """
    improvements = []
    
    # Process from largest to smallest
    for layout_id in sorted(dict_of_tree_list.keys(), key=lambda x: int(x), reverse=True):
        layout = dict_of_tree_list[layout_id]
        n = int(layout_id)
        
        if n < 2:  # Skip N=1
            continue
            
        # Get bounding box corners
        all_polygons = [t.polygon for t in layout]
        bounds = unary_union(all_polygons).bounds
        
        corners = [
            (bounds[0], bounds[1]),  # bottom-left
            (bounds[0], bounds[3]),  # top-left
            (bounds[2], bounds[1]),  # bottom-right
            (bounds[2], bounds[3]),  # top-right
        ]
        
        for corner_x, corner_y in corners:
            # Create dictionary: distance -> tree
            # Use max distance from corner to any edge of tree's bounding box
            candidates = {}
            for tree in layout:
                dist = max(
                    abs(tree.polygon.bounds[0] - corner_x),
                    abs(tree.polygon.bounds[2] - corner_x),
                    abs(tree.polygon.bounds[1] - corner_y),
                    abs(tree.polygon.bounds[3] - corner_y),
                )
                candidates[dist] = tree
            
            # Sort by distance (closest first)
            candidates = dict(sorted(candidates.items()))
            
            # Reconstruct by order and compare with existing
            layout_new = []
            for tree in candidates.values():
                layout_new.append(tree)
                layout_new_size = len(layout_new)
                layout_new_size_key = f'{layout_new_size:03d}'
                
                if layout_new_size_key not in dict_of_side_length:
                    continue
                    
                best_side = dict_of_side_length[layout_new_size_key]
                candidate_side = get_tree_list_side_length(layout_new)
                
                if candidate_side < best_side:
                    improvement = float(best_side - candidate_side)
                    if verbose:
                        print(f'IMPROVEMENT! N={layout_new_size} from layout {n}: '
                              f'{float(best_side):.8f} -> {float(candidate_side):.8f} '
                              f'(saved {improvement:.8f})')
                    
                    dict_of_tree_list[layout_new_size_key] = copy_layout(layout_new)
                    dict_of_side_length[layout_new_size_key] = candidate_side
                    improvements.append((layout_new_size, improvement))
    
    return improvements

print("Rebuild function defined.")

Rebuild function defined.


In [5]:
# Run the rebuild from corners technique
print("Running rebuild from corners...")
print("="*60)

improvements = rebuild_from_corners(dict_of_tree_list, dict_of_side_length)

print("="*60)
print(f"Total improvements found: {len(improvements)}")

if improvements:
    total_saved = sum(imp[1] for imp in improvements)
    print(f"Total side length saved: {total_saved:.8f}")
    print("\nImprovements by N:")
    for n, saved in sorted(improvements):
        print(f"  N={n}: saved {saved:.8f}")

Running rebuild from corners...


IMPROVEMENT! N=65 from layout 101: 4.86277400 -> 4.85966054 (saved 0.00311346)


Total improvements found: 1
Total side length saved: 0.00311346

Improvements by N:
  N=65: saved 0.00311346


In [6]:
# Calculate new score
new_score = get_total_score(dict_of_side_length)
print(f"\nBaseline score: {float(baseline_score):.6f}")
print(f"New score: {float(new_score):.6f}")
print(f"Improvement: {float(baseline_score - new_score):.6f}")


Baseline score: 70.659958
New score: 70.659493
Improvement: 0.000466


In [7]:
# Check for overlaps in improved solutions
def check_overlaps(tree_list):
    """Check if any trees overlap."""
    polygons = [t.polygon for t in tree_list]
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > 1e-10:
                    return True, i, j, intersection.area
    return False, None, None, 0

print("Checking for overlaps in all solutions...")
overlap_found = False
for n in range(1, 201):
    key = f'{n:03d}'
    if key in dict_of_tree_list:
        has_overlap, i, j, area = check_overlaps(dict_of_tree_list[key])
        if has_overlap:
            print(f"N={n}: OVERLAP between trees {i} and {j}, area={area}")
            overlap_found = True

if not overlap_found:
    print("No overlaps found! Solution is valid.")

Checking for overlaps in all solutions...


No overlaps found! Solution is valid.


In [8]:
# Save the improved solution
tree_data = []
for group_name in sorted(dict_of_tree_list.keys(), key=lambda x: int(x)):
    tree_list = dict_of_tree_list[group_name]
    for item_id, tree in enumerate(tree_list):
        tree_data.append({
            'id': f'{group_name}_{item_id}',
            'x': f's{tree.center_x}',
            'y': f's{tree.center_y}',
            'deg': f's{tree.angle}'
        })

df_new = pd.DataFrame(tree_data)
df_new.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved improved solution to /home/submission/submission.csv")
print(f"Total rows: {len(df_new)}")

Saved improved solution to /home/submission/submission.csv
Total rows: 20100


In [9]:
# Save metrics
import json

metrics = {
    'cv_score': float(new_score),
    'baseline_score': float(baseline_score),
    'improvement': float(baseline_score - new_score),
    'num_improvements': len(improvements),
    'model_type': 'rebuild_from_corners',
    'notes': 'Rebuild from corners technique from chistyakov kernel'
}

with open('/home/code/experiments/002_rebuild_from_corners/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nFinal CV Score: {float(new_score):.6f}")


Final CV Score: 70.659493


In [10]:
# Run multiple iterations to find more improvements\nprint("Running multiple iterations of rebuild from corners...\\n")\n\ntotal_improvements = []\nfor iteration in range(5):\n    print(f"\\n=== Iteration {iteration + 1} ===\")\n    improvements = rebuild_from_corners(dict_of_tree_list, dict_of_side_length, verbose=True)\n    if not improvements:\n        print("No more improvements found. Stopping.")\n        break\n    total_improvements.extend(improvements)\n\nprint(f"\\n\\nTotal improvements across all iterations: {len(total_improvements)}")\n\n# Calculate final score\nfinal_score = get_total_score(dict_of_side_length)\nprint(f"\\nFinal score: {float(final_score):.6f}")\nprint(f"Total improvement from baseline: {float(baseline_score - final_score):.6f}")

In [ ]:
# Check current score\ncurrent_score = get_total_score(dict_of_side_length)\nprint(f\"Current score: {float(current_score):.6f}\")\nprint(f\"Improvement from baseline: {float(baseline_score - current_score):.6f}\")\n\n# Save the solution\ntree_data = []\nfor group_name in sorted(dict_of_tree_list.keys(), key=lambda x: int(x)):\n    tree_list = dict_of_tree_list[group_name]\n    for item_id, tree in enumerate(tree_list):\n        tree_data.append({\n            'id': f'{group_name}_{item_id}',\n            'x': f's{tree.center_x}',\n            'y': f's{tree.center_y}',\n            'deg': f's{tree.angle}'\n        })\n\ndf_new = pd.DataFrame(tree_data)\ndf_new.to_csv('/home/submission/submission.csv', index=False)\nprint(f\"Saved to /home/submission/submission.csv\")"